In [ ]:
import sys
import shutil
sys.path.append('../dependencies/')
import pyemu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import pathlib as pl
import os, platform

In [ ]:
pst_name='./MASTER_SEN/global_sens.pst'

In [ ]:
pst = pyemu.Pst(pst_name)

In [ ]:
morris_sum_file = pst_name.replace('.pst','.msn')

In [ ]:
morris_df = pd.read_csv(morris_sum_file, index_col=0)

In [ ]:
ax = morris_df.sort_values(by='sen_mean_abs',ascending=False).iloc[:25].sen_mean_abs.apply(np.log10).plot.bar(figsize=(7,7))
